# Loadind libaries and data

In [3]:
!pip install -q avro fastavro faker geopy pycountry requests beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 54.3 MB/s eta 0:00:00


In [9]:
import random
import csv
import time
import fastavro
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import avro.schema
from avro.datafile import DataFileWriter
from avro.io import DatumWriter
import fastavro
from fastavro import writer, reader, parse_schema
import io
import pandas as pd
from faker import Faker
from datetime import datetime
import pytz
import pycountry
import requests
from bs4 import BeautifulSoup

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#folder_path = '/content/drive/MyDrive/Stream_Analytics/Deliverable1/'

folder_path = '/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/'

df = pd.read_csv(folder_path + 'Spotify-2000.csv')
df.head(10)

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4,76
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256,1,3,59
5,6,The Road Ahead (Miles Of The Unknown),City To City,alternative pop rock,2004,99,46,54,-9,14,14,247,0,2,45
6,7,She Will Be Loved,Maroon 5,pop,2002,102,71,71,-6,13,54,257,6,3,74
7,8,Knights of Cydonia,Muse,modern rock,2006,137,96,37,-5,12,21,366,0,14,69
8,9,Mr. Brightside,The Killers,modern rock,2004,148,92,36,-4,10,23,223,0,8,77
9,10,Without Me,Eminem,detroit hip hop,2002,112,67,91,-3,24,66,290,0,7,82


# Creating the schema

In [13]:
# set a timezone-aware start time using UTC timezone
utc = pytz.utc
start_time = datetime(2020, 1, 1, tzinfo=utc)

# AVRO Schema

spotify_wrap_schema = {

  "doc": "User listening history with song details.", # documentation/description of schema

  "name": "ListeningHistory", # name of record type the schema represents

  "namespace": "music", # helps to prevent name clashes with other schema

  "type": "record", # type of schema - record representing a complex structure

  # fields of the recod
  "fields": [
    {"name": "user_id", "type": "int"},

    {
      "name": "user_location",
      "type": {
        "type": "record",
        "name": "Location",
        "fields": [
          {"name": "latitude", "type": "float"},
          {"name": "longitude", "type": "float"},
          {"name": "country", "type": "string"},
          {"name": "city", "type": "string"},
          {"name": "timezone", "type": "string"}
        ]
      }
    },


    {"name": "interaction", "type": "string"},

    {"name": "song_name", "type": "string"},

    {"name": "artist", "type": "string"},

    {"name": "genre", "type": "string"},

    {"name": "year", "type": "int"},

    {"name": "popularity", "type": "int"},

    {"name": "danceability", "type": "float"},

    {"name": "energy", "type": "float"},

    {"name": "length_seconds", "type": "int"},

    {
      "name": "listening_time",
      "type": {
        "type": "record",
        "name": "ListeningPeriod",
        "fields": [
            {"name": "start_time", "type": "long"},
            {"name": "end_time", "type": "long"}
        ]
    }
    },

    {
      "name": "personality",
      "type": "string",
    }
  ]
}

# Initialization

In [14]:
personalities = {
    "The early adopter": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Variety", "C/U": "Commonality"},
    "The nomad": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The specialist": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Variety", "C/U": "Uniqueness"},
    "The enthusiast": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The connoisseur": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The deep diver": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Uniqueness"},
    "The fanclubber": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Variety", "C/U": "Commonality"},
    "The top charter": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The replayer": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The jukeboxer": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Commonality"},
    "The voyager": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The devotee": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The maverick": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The time traveler": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Uniqueness"},
    "The musicologist": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Commonality"},
    "The adventurer": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Variety", "C/U": "Uniqueness"}
}

song_df = df[['Title', 'Artist', 'Top Genre', 'Year', 'Popularity', 'Length (Duration)']]
fake = Faker()
user_song_history = {}
records = []
user_country_mapping = {}

# Functions

In [15]:
def dynamic_song_selection(user_id, personality, history, song_df):
    year_preference = random.choice([1990, 2000, 2010, 2020])
    filtered_df = song_df[song_df['Year'] == year_preference]

    if user_id in history:
        filtered_df = filtered_df[~filtered_df['Title'].isin(history[user_id])]

    if filtered_df.empty:
        filtered_df = song_df

    return filtered_df.sample(n=1)

def simulate_interaction():
    interactions = ['played', 'paused', 'skipped', 'repeated', 'added_to_playlist']
    probabilities = [0.5, 0.2, 0.15, 0.1, 0.05]
    return random.choices(interactions, weights=probabilities, k=1)[0]

In [16]:
#This is same as dynamic song selection but takes into account personality
def dynamic_song_selection_2(user_id, personality, user_song_history, df):
    characteristics = personalities[personality]
    feature = random.choice(list(characteristics.keys()))
    value = characteristics[feature]

    if user_id not in user_song_history or not user_song_history[user_id]:
        # If the user has no song history, select a random song based on artist
        random_song = df[df['Artist'].isin(df['Artist'].unique())].sample(n=1)
        user_song_history[user_id] = [random_song]

        return random_song

    # User has song history, apply filtering logic based on personality
    if feature == "F/E":
        filter_condition = df['Artist'].isin(user_song_history[user_id][0]['Artist']) if value == "Familiarity" else ~df['Artist'].isin(user_song_history[user_id][0]['Artist'])
    elif feature == "T/N":
        filter_condition = df['Year'] == user_song_history[user_id][0]['Year'].values[0] if value == "Newness" else ~(df['Year'] == user_song_history[user_id][0]['Year'].values[0])
    elif feature == "L/V":
        filter_condition = df['Title'].isin(user_song_history[user_id][0]['Title']) if value == "Loyalty" else ~df['Title'].isin(user_song_history[user_id][0]['Title'])
    elif feature == "C/U":
        filter_condition = df['Popularity'] > user_song_history[user_id][0]['Popularity'].values[0] if value == "Commonality" else df['Popularity'] < user_song_history[user_id][0]['Popularity'].values[0]

    filtered_df = df[filter_condition].sample(n=1)
    user_song_history[user_id].append(filtered_df)

    # Return the same structure as dynamic_song_selection
    return filtered_df.sample(n=1)



In [17]:
def parse_duration(duration_str):
    # Remove commas and any other non-numeric characters
    duration_str = ''.join(c for c in duration_str if c.isdigit() or c == ':')

    # Split the string into minutes and seconds
    parts = duration_str.split(':')

    if len(parts) == 2:
        minutes, seconds = map(int, parts)
    elif len(parts) == 1:
        minutes = 0
        seconds = int(parts[0])
    else:
        raise ValueError("Invalid duration format: " + duration_str)

    return minutes * 60 + seconds

In [18]:
def generate_valid_country_code():
    while True:
        country_code = fake.country_code(representation="alpha-2")
        # Ensure the country code is compatible with local_latlng
        if fake.local_latlng(country_code=country_code):
            return country_code

In [19]:
def generate_user_location(user_id,user_country_mapping):
    if user_id in user_country_mapping:
        # If the user already has a country assigned, use that country
        random_country = user_country_mapping[user_id]
    else:
        # Generate a new country for the user
        random_country = generate_valid_country_code()
        user_country_mapping[user_id] = random_country

    location = fake.local_latlng(country_code=random_country)
    latitude, longitude, city, country, timezone = location

    return {
        "user_id": user_id,
        "latitude": latitude,
        "longitude": longitude,
        "country": country,
        "city": city,
        "timezone": timezone
    }

#OLD FUNCTION
#def generate_user_location():


    # Extract locales available
#    country_code_options = get_faker_locales()
#    #country_code_options = ['US', 'GB', 'AU', 'IN', 'IE']#
#    random_country = random.choice(country_code_options)

#    location = fake.local_latlng(country_code=random_country)



#    return {
#        "latitude": location[0],
#        "longitude": location[1],
#        "country": random_country,
#        "city": location[2],
#        "timezone": location[4]  # Initialize timezone
#    }

In [20]:
def generate_record(user_id, personality, df, user_location, start_time, n_records, n_records2):

    user_song_history.setdefault(user_id, [])
    characteristics = personalities[personality]
    feature, value = random.choice(list(characteristics.items()))

    # Refactor personality-based filtering into a function
    def filter_songs(df, feature, value, history):
        if feature == "F/E":
            exploration_threshold = 0.8
            is_exploring = random.random() > exploration_threshold if value == "Exploration" else random.random() < exploration_threshold
            artist_in_history = df['Artist'].isin(history[user_id]) if user_id in history else pd.Series([False] * len(df))
            return df[artist_in_history] if is_exploring else df[~artist_in_history]
        elif feature == "T/N":
            newness_threshold = 0.8
            is_new = random.random() < newness_threshold if value == "Newness" else random.random() > newness_threshold
            return df[df['Year'] == 2019] if is_new else df[df['Year'] != 2019]
        elif feature == "L/V":
            loyalty_threshold = 0.8
            is_loyal = random.random() < loyalty_threshold if value == "Loyalty" else random.random() > loyalty_threshold
            title_in_history = df['Title'].isin(history[user_id]) if user_id in history else pd.Series([False] * len(df))
            return df[title_in_history] if is_loyal else df[~title_in_history]
        elif feature == "C/U":
            commonality_threshold = 0.8
            is_common = random.random() < commonality_threshold if value == "Commonality" else random.random() > commonality_threshold
            return df[df['Popularity'] > 70] if is_common else df[df['Popularity'] <= 70]
        return df

    filtered_df = filter_songs(df, feature, value, user_song_history)

    if filtered_df.empty:
        filtered_df = df

    song_row = filtered_df.sample(n=1).iloc[0]
    user_song_history[user_id].append(song_row['Title'])

    # Time adjustments
    if start_time is not None:
        n_records += 1
        n_records2 += 1
        if n_records > 20:
            start_time += timedelta(days=1)
            n_records = 0
        if n_records2 > 400:
            start_time += relativedelta(months=1)
            n_records2 = 0

    # Interaction handling
    interaction = random.choices(['played', 'paused', 'shuffle'], weights=[0.7, 0.2, 0.1], k=1)[0]
    interaction_duration = random.randint(30, 210) if interaction != 'paused' else random.randint(30, 90)
    end_time = start_time + timedelta(seconds=interaction_duration)

    # Format time consistently
    start_time_ms = int(record["listening_time"]["start_time"].timestamp() * 1000)
    end_time_ms = int(record["listening_time"]["end_time"].timestamp() * 1000)
    #start_time_str = start_time.strftime("%Y-%m-%d %H:%M:%S") - does not work bc avro schema says we use a timestamp, not this format
    #end_time_str = end_time.strftime("%Y-%m-%d %H:%M:%S")


    # Return structured record
    return {
        "user_id": user_id,
        "user_location": user_location,
        "interaction": interaction,
        "song_name": song_row['Title'],
        "artist": song_row['Artist'],
        "genre": song_row['Top Genre'],
        "year": int(song_row['Year']),
        "popularity": int(song_row['Popularity']),
        "danceability": float(song_row.get('Danceability', 0)),  # Assuming these fields are now included and handled as floats
        "energy": float(song_row.get('Energy', 0)),
        "length_seconds": int(song_row['Length (Duration)']),
        "listening_time": (start_time_ms, end_time_ms),
        "personality": personality
    }



In [21]:
def simulate_user_session(user_id, start_time, duration_hours, personality, user_song_history, song_df,user_country_mapping):
    session_end_time = start_time + timedelta(hours=duration_hours)
    current_time = start_time
    session_records = []
    user_location = generate_user_location(user_id, user_country_mapping)

    while current_time < session_end_time:
        song_row = dynamic_song_selection(user_id, personality, user_song_history, song_df).iloc[0]
        interaction = simulate_interaction()

        if interaction == 'skipped':
            listen_duration = random.randint(5, 30)
        elif interaction == 'repeated':
            listen_duration = random.randint(360, 600)
        else:
            listen_duration = random.randint(180, 300)

        end_time = current_time + timedelta(seconds=listen_duration)

        start_time_ms = int(start_time.timestamp() * 1000)
        end_time_ms = int(session_end_time.timestamp() * 1000)

            # Generate realistic user location
        user_location = generate_user_location(user_id, user_country_mapping)


        record = {
            "user_id": user_id,
            "user_location": user_location,
            "interaction": interaction,
            "song_name": song_row['Title'],
            "artist": song_row['Artist'],
            "genre": song_row['Top Genre'],
            "year": song_row['Year'],
            "popularity": song_row['Popularity'],
            "danceability": song_row.get('Danceability', 0),
            "energy": song_row.get('Energy', 0),
            "length_seconds": parse_duration(song_row['Length (Duration)']),
            "listening_time": {"start_time": start_time_ms, "end_time": end_time_ms},
            "personality": personality
        }
        session_records.append(record)

        if user_id not in user_song_history:
            user_song_history[user_id] = []
        user_song_history[user_id].append(song_row['Title'])

        current_time = end_time

    return session_records


#for user_id in range(1, 11):  #simulating 100 users
#    for _ in range(random.randint(1, 3)):  #assuming each user has 1-3 sessions
#        start_time = fake.date_time_between(start_date="-1y", end_date="now")
#        current_time = start_time
#        duration_hours = random.randint(1, 3)  #session last between 1 and 3 hours
#        personality = random.choice(list(personalities.keys()))
#        session_records = simulate_user_session(user_id, current_time, duration_hours, personality, user_song_history, song_df)
#        records.extend(session_records)


In [22]:
def main(num_sessions_per_user,session_duration_hours_range,n_users):
  for user_id in range(1,n_users):  # Example user IDs from 1 to 1000
    personality = random.choice(list(personalities.keys()))
    user_sessions_records = []
    for _ in range(num_sessions_per_user):
        start_time = fake.date_time_between(start_date="-1y", end_date="now", tzinfo=utc)
        session_duration_hours = random.randint(*session_duration_hours_range)
        # Ensure simulate_user_session or your equivalent function makes use of the new generate_record function
        session_records = simulate_user_session(user_id, start_time, session_duration_hours, personality, user_song_history, song_df, user_country_mapping)
        user_sessions_records.extend(session_records)
    records.extend(user_sessions_records)

  # Serialize data
  with io.BytesIO() as bytes_writer:
      fastavro.writer(bytes_writer, spotify_wrap_schema, records)
      serialized_data = bytes_writer.getvalue()

  # Output the serialized data length
  print("Serialized data length:", len(serialized_data))

  # If you want to save the serialized data to a file
  with open('spotify_wrapped_data.avro', 'wb') as avro_file:
      avro_file.write(serialized_data)

# Running the code

In [23]:
num_sessions_per_user = 3
session_duration_hours_range = (1, 3)

for user_id in range(1, 12):  # Example user IDs from 1 to 1000
    personality = random.choice(list(personalities.keys()))
    user_sessions_records = []
    for _ in range(num_sessions_per_user):
        start_time = fake.date_time_between(start_date="-1y", end_date="now", tzinfo=utc)
        session_duration_hours = random.randint(*session_duration_hours_range)
        # Ensure simulate_user_session or your equivalent function makes use of the new generate_record function
        session_records = simulate_user_session(user_id, start_time, session_duration_hours, personality, user_song_history, song_df, user_country_mapping)
        user_sessions_records.extend(session_records)
    records.extend(user_sessions_records)

In [24]:
# Serialize data
with io.BytesIO() as bytes_writer:
    fastavro.writer(bytes_writer, spotify_wrap_schema, records)
    serialized_data = bytes_writer.getvalue()

# Output the serialized data length
print("Serialized data length:", len(serialized_data))

# If you want to save the serialized data to a file
with open('spotify_wrapped_data.avro', 'wb') as avro_file:
    avro_file.write(serialized_data)

Serialized data length: 118113


In [26]:
#RUN THIS TO DO ALL IN ONE GO
main(num_sessions_per_user=3,session_duration_hours_range=(1,3),n_users=100)

Serialized data length: 1342036
